---
layout: post  
mathjax: true  
comments: true  
title: DICOM Processing  
tags: [Computer Vision]  
---  
What is the DICOM file format and how does it relate to computer vision?  

A little while ago I decided to undertake a computer vision project to detect abnormalities in brain CT scans by utilizing 3-dimensional [convolutional neural networks](https://jason-adam.github.io/convolutions/). Admittedly, this was a little more ambitious than I had time for, but it was a great learnring experience, and it forced me to work on my pre-processing skills due to the nature of the file formats that certain medical images come in. This leads me to the DICOM file format for medical imaging.   
## What is a DICOM file?  

[DICOM](https://en.wikipedia.org/wiki/DICOM) stands *Digital Imaging and Communications in Medicine*, and is considered the standard for communication and management of medical imaging information and related data. The most common usage for DICOM is to store and transmit medical images.  

Now that we have a very cursory understanding of what DICOM is and what it's used for, lets start on our pre-processing pipeline for these medical images.  

## Imports

In [21]:
import os
from collections import Counter
from typing import List, Tuple

import numpy as np
import pydicom
import scipy.ndimage

%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [7]:
# GLOBAL VARIABLES
INPUT_FOLDER = "../imgs/2020-01-12-dicom/sample_dicom/Unknown Study/"

## Obtaining the Data  
Before jumping into the code, I want to take a quick step back. The original study where I obtained the data can be found [here](http://headctstudy.qure.ai/#dataset). The original study predicted multiple labels for the patient (i.e. intracranial hemorrhage, fracture, midline shifts, etc.), but for the simplification of my project, I selected the abnormality affecting the most scans in the sample dataset I downloaded (Intracranial Hemorrhage).  

Below is a simple way to download one zip folder for a patient with their scans. If you want to download all the zip folders, I recommend giving yourself some time, as there are almost 500 zip files, each containing a full set (sometimes multiple sets) of scans for a patient. The total size of all patients downloaded and unzipped is roughly 40-50GB.

After the scans have downloaded, we can use the following code to see which size of scans (number of slices is the most prominent (they come in various slices). The function simply loops through our scan folders and counts up the amount of slices into a list of tuples using the `Counter` object.

In [17]:
def most_common_slice_count() -> List[Tuple]:
    slice_counts: list = []
    list_scans = os.listdir(INPUT_FOLDER)

    for scan_folders in list_scans:
        slice_path = os.path.join(INPUT_FOLDER, scan_folders + "/")
        slices = os.listdir(slice_path)
        slice_counts.append(len(slices))

    counts = Counter(slice_counts)
    return counts.most_common(1)


print(most_common_slice_count())

[(239, 3)]


## Building the Pipeline  
For this example, we only have one patient, but their folder contains three sets of scans all consisting of 239 slices. The next function using the `pydicom` library which was created for dealing with DICOM image files. The code takes in an argument for the size of slices to retrieve (239 in our case). This is a generator function which means that it returns an iterator. If you're unfamiliar with the concept, there is a great guide [here](https://realpython.com/introduction-to-python-generators/). The code reads in all the scans using the `pydicom.read_file` function and loads them to a list.

In [23]:
def get_loaded_scans(base_path: str, slice_count: int):
    list_scans = os.listdir(INPUT_FOLDER)

    for scan_folders in list_scans:
        slice_path = os.path.join(INPUT_FOLDER, scan_folders + "/")
        slices = os.listdir(slice_path)
        if len(slices) != slice_count:
            continue
        else:
            slices.sort()
            yield list(
                [pydicom.read_file(os.path.join(slice_path, s)) for s in slices]
            )

We can test to see if our generator function works:

In [24]:
test_scans = get_loaded_scans(INPUT_FOLDER, slice_count=239)

for scan in test_scans:
    print(len(scan))

239
239
239


We can see that we did yield three sets of scans of length 239! It looks like our generator function is working as intended. Before we look at the next function, there is some additional information regarding CT scans that is relevant. The pixels in a CT scan are represented in [Hounsfield Units](https://en.wikipedia.org/wiki/Hounsfield_scale). The Hounsfield scale is a quantitative scale for describing radiodensity, and the value ranges represent different types of tissue. Below is an image of the value ranges for context.  

![](../imgs/2020-01-12-dicom/hounsfield_scale.jpg)  

Prior to looking at any scans, we can guess that we'll see several of these ranges, including bone, grey matter, etc.  

Now that we have our intial generator function setup, I'm going to show what's actually in a DICOM file prior to skipping to extracting the pixels. Let's read in one for our example.

In [30]:
sample_slice = pydicom.read_file(
    "../imgs/2020-01-12-dicom/sample_dicom/Unknown Study/CT 4cc sec 150cc D3D on/CT000082.dcm"
)

print(sample_slice)

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL']
(0008, 0012) Instance Creation Date              DA: ''
(0008, 0013) Instance Creation Time              TM: ''
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.276.0.7230010.3.1.4.296485376.1.1521714308.2032317
(0008, 0020) Study Date                          DA: ''
(0008, 0023) Content Date                        DA: ''
(0008, 0030) Study Time                          TM: ''
(0008, 0033) Content Time                        TM: ''
(0008, 0050) Accession Number                    SH: ''
(0008, 0060) Modality                            CS: 'CT'
(0008, 0070) Manufacturer                        LO: ''
(0008, 0090) Referring Physician's Name          PN: ''
(0008, 103e) Series Description                  LO: '4cc sec 150cc D3D on'
(0008, 1090) Manufacturer's Mode

We can see that the DICOM slice contains very rich metadata about the patient and the slice itself. The final element in the slice contains an array of elements. These are our pixels.  

The following function was adapted and modified from [here](https://www.kaggle.com/saptarc/processing-lungs-ct-scans). It stacks all scans into a 3D numpy array, adjusts pixels based on a padding value, and converts to hounsfield units. The results is a transformed 3D numpy array. Since this function accepts a list (an iterator), we can pass the output of our previous function to it. This function also is a generator (it returns an iterator). We can begin to see why they are so powerful for preprocessing pipelines. The resultant iterator is only loaded into memory one at a time instead of the entire batch. This makes these pipelines extremely fast and memory efficient.

In [54]:
def get_pixels_hu(slices: list):
    for scans in slices:
        try:
            patient_id = str(scans[0].PatientName)
            image = np.stack([s.pixel_array for s in scans])
            image = image.astype(np.float)

            # Set outside-of-scan pixels to 0
            image[image == -2000] = 0

            # Convert to Hounsfield units (HU)
            for slice_number in range(len(scans)):

                intercept = scans[slice_number].RescaleIntercept
                slope = scans[slice_number].RescaleSlope

                if slope != 1:
                    image[slice_number] = slope * image[slice_number]

                image[slice_number] += np.float(intercept)

            yield np.array(image, dtype=np.float), patient_id
        except:
            pass

Our final function in the pipleine is a normalization function.

In [55]:
def normalize_stacks(image, min_bound=-1000.0, max_bound=2000.0):
    image = (image - min_bound) / (max_bound - min_bound)
    image[image > 1] = 1
    image[image < 0] = 0
    return image

Below is a simple sample of our total pipeline that was in the main function.

In [57]:
test_scans = get_loaded_scans(INPUT_FOLDER, slice_count=239)
scan_array = get_pixels_hu(test_scans)

for i, j in scan_array:
    normalized = scipy.ndimage.interpolation.zoom(i, (1, 0.50, 0.50), mode="nearest")
    normalized = normalize_stacks(normalized)
    normalized = normalized - np.mean(normalized)
    print(normalized.shape, j)